In [ ]:
import os
import subprocess
import multiprocessing
import itertools
import pandas as pd
from scipy import stats

path= 'images'
makeTypes = ['o0-masks', 'o0-shifts', 'o3-masks', 'o3-shifts']
languages = ['c', 'asm1', 'asm2']
filters = ['hsl', 'blur', 'merge']
command = {
    'blur': './runTests {language} {filter} "{src}" "{dst}"',
    'merge': './runTests {language} {filter} "{src}" "{src2}" "{dst}" {alpha}',
    'hsl': './runTests {language} {filter} "{src}" "{dst}" {h} {s} {l}'
}

trim = 0.2
opt = 0
hhssll = [(0.0, 0.0, 0.0), (99.0, 0.0, 0.0)]
alpha = [0.0, 0.14674845465465456489765465478976546548]

def worker(filt, lang, dataset):
    output = []
    rpath = os.path.join(path, filt)

    for img in os.listdir(rpath):
        rimg = os.path.join(rpath, img)

        if os.path.isfile(rimg):
            
            if filt == 'hsl':
                for (h, s, l) in hhssll:
                    comm = command[filt].format(
                            language = lang,
                            filter = filt,
                            src = rimg,
                            dst = './current.bmp',
                            h = h,
                            s = s,
                            l = l
                        )

                    delta = subprocess.check_output(comm, shell=True)
                    delta = list(delta.strip())
                    rdelta = pd.Series(delta)

                    output.append({
                        'filename': img,
                        'language': lang,
                        'filter': filt,
                        'deltas': delta,
                        'trimmed_mean': stats.trim_mean(rdelta, trim),
                        'mean': rdelta.mean(),
                        'variance': rdelta.var(),
                        'optimization': opt,
                        'h': h,
                        's': s,
                        'l': l
                    })
            elif filt == 'blur':
                comm = command[filt].format(
                        language = lang,
                        filter = filt,
                        src = rimg,
                        dst = './current.bmp'
                    )

                delta = subprocess.check_output(comm, shell=True)
                delta = list(delta.strip())
                rdelta = pd.Series(delta)

                output.append({
                    'filename': img,
                    'language': lang,
                    'filter': filt,
                    'deltas': delta,
                    'trimmed_mean': stats.trim_mean(rdelta, trim),
                    'mean': rdelta.mean(),
                    'variance': rdelta.var(),
                    'optimization': opt
                })
            elif filt == 'merge':
                for a in alpha:
                    comm = command[filt].format(
                            language = lang,
                            filter = filt,
                            src = rimg,
                            src2 = rimg,
                            dst = './current.bmp',
                            alpha = a
                        )

                    delta = subprocess.check_output(comm, shell=True)
                    delta = list(delta.strip())
                    rdelta = pd.Series(delta)

                    output.append({
                        'filename': img,
                        'language': lang,
                        'filter': filt,
                        'deltas': delta,
                        'trimmed_mean': stats.trim_mean(rdelta, trim),
                        'mean': rdelta.mean(),
                        'variance': rdelta.var(),
                        'optimization': opt,
                        'alpha': a
                    })
                    
    dataset.append(output)

def runExperiment(filters, languages):
    jobs = []
    manager = multiprocessing.Manager()
    dataset = manager.list([])

    for filt, lang in itertools.product(filters, languages):
        p = multiprocessing.Process(target=worker, args=(filt, lang, dataset))
        jobs.append(p)
        p.start()

    for job in jobs:
        job.join()
        
    data = []

    for lst in dataset:
        data += lst
    
    return data

def enrich(data):
    dst = pd.DataFrame(data)
    dst.set_index(['filter', 'language'])
    dst['experiment'] = dst['filename'].apply(lambda x: x.split(' ')[0])
    dst['width'] = dst['filename'].apply(lambda x: x.split(' ')[1].split('x')[0])
    dst['height'] = dst['filename'].apply(lambda x: x.split(' ')[1].split('x')[1])
    dst['id'] = dst['filename'].apply(lambda x: x.split(' ')[2].replace('(', '').replace(')', '').replace('.bmp', ''))
    return dst